In [3]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [45]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [46]:
import nltk.corpus
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [58]:
vectorizer = CountVectorizer(ngram_range=(1,2),token_pattern=r'\b\w+\b',min_df=0)
tfTransformer = TfidfTransformer(use_idf = True)

In [48]:
import string
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(string.punctuation)
stopwords.remove('not')
lemmatizer = WordNetLemmatizer()


In [49]:
def cleanText(summary):
    summary = summary.replace('*','').replace('-',' ').replace('/',' ').replace("'",' ')
    tokens_summary = [str.lower().strip(string.punctuation) for str in summary.split() if str not in stopwords]
    lemma_summary = [lemmatizer.lemmatize(token) for token in tokens_summary if len(token) > 0]
    #for word in lemma_summary:
    #    wordList.append(word)
    return(' '.join(word for word in lemma_summary))
    

In [50]:
words = []
classes = []
documents = []

# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        documents.append((pattern, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents
9 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
0 unique stemmed words []


In [59]:
X = vectorizer.fit_transform([pattern for pattern,tag in documents])
Y = tfTransformer.fit_transform(X)

In [67]:
output = []
output_empty = [0] * len(classes)
for doc in documents:
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    output.append(output_row)
len(output)

27

In [68]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(vectorizer.get_feature_names())])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(classes), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)

model.fit(Y.toarray(), output, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.17874 | time: 0.011s
| Adam | epoch: 1000 | loss: 0.17874 - acc: 0.9911 -- iter: 24/27
Training Step: 4000  | total loss: 0.17101 | time: 0.013s
| Adam | epoch: 1000 | loss: 0.17101 - acc: 0.9920 -- iter: 27/27
--
INFO:tensorflow:E:\USF\Courses\ISM6930-DSP\Projects\ResumeBot\code\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [107]:
test_X = vectorizer.transform(["I'd like to rent a moped"])
test_Y = tfTransformer.transform(test_X)
ft = vectorizer.get_feature_names() 


result = list(map(lambda row:dict(zip(ft,row)),test_Y.toarray()))
#print(list(result[0].values()))
result_array = model.predict([list(result[0].values())]).tolist()[0]
result_array

[0.026589520275592804,
 0.025932131335139275,
 0.002491482999175787,
 1.3776259038422722e-05,
 0.03694448247551918,
 0.0099998340010643,
 0.8976433277130127,
 6.531870167236775e-05,
 0.00032017339253798127]

In [108]:

classes[result_array.index(max(result_array))]

'rental'

In [106]:
max(result_array)

0.9329773187637329